# Preprocessing data Youtube Rewind 2023

#### Import library

In [1]:
from dotenv import load_dotenv
from pymongo import MongoClient
import os
import pandas as pd
import re

In [2]:
# Memuat value dari file .env
load_dotenv()

mongodb_url = os.getenv('URL_SANDY')
local_url = os.getenv('URL_LOCAL')

In [3]:
database_name = "youtube_rewind_indonesia"
collection_from = "cleansing_2023"
collection_name = "preprocessing_2023"

#### Import dataset dari MongoDB

In [4]:
# Koneksi ke MongoDB
client = MongoClient(local_url)

# Pilih database dan koleksi
db = client[database_name]
collection = db[collection_from]

# Query data dari koleksi
cursor = collection.find()

# Konversi cursor ke list dan kemudian ke DataFrame
data = list(cursor)
df = pd.DataFrame(data)

# Menutup koneksi
client.close()

In [5]:
df

,_id,textOriginal
0,665167dbc7b93965e002c573,Selamat menikmati REWIND INDONESIA di jam 19.4...
1,665167dbc7b93965e002c574,KERENN!!!\nMulai di menit ke 8 sampai akhir fi...
2,665167dbc7b93965e002c575,4.24 itu bukannya bg tanboy ya?
3,665167dbc7b93965e002c576,this is so grazy!!!:'''(
4,665167dbc7b93965e002c577,"2024 baru nonton🤚 kukira ada ""beras rose brand"""
...,...,...
34325,66516849c7b93965e0035bd1,Keren banget mantap ... Saking kerennya menuru...
34326,66516849c7b93965e0035bd2,kereeennn paarrraahgg
34327,66516849c7b93965e0035bd3,Kok ga ada om deddy corbuzier?
34328,66516849c7b93965e0035bd5,Pas nyanyi maju tak gentar merinding banget si...


#### Mengubah data text menjadi lowercase

In [6]:
df['textOriginal'] = df['textOriginal'].str.lower()

df

,_id,textOriginal
0,665167dbc7b93965e002c573,selamat menikmati rewind indonesia di jam 19.4...
1,665167dbc7b93965e002c574,kerenn!!!\nmulai di menit ke 8 sampai akhir fi...
2,665167dbc7b93965e002c575,4.24 itu bukannya bg tanboy ya?
3,665167dbc7b93965e002c576,this is so grazy!!!:'''(
4,665167dbc7b93965e002c577,"2024 baru nonton🤚 kukira ada ""beras rose brand"""
...,...,...
34325,66516849c7b93965e0035bd1,keren banget mantap ... saking kerennya menuru...
34326,66516849c7b93965e0035bd2,kereeennn paarrraahgg
34327,66516849c7b93965e0035bd3,kok ga ada om deddy corbuzier?
34328,66516849c7b93965e0035bd5,pas nyanyi maju tak gentar merinding banget si...


#### Menghapus data dengan atribut text null

In [7]:
df = df[df['textOriginal'] != '']
df

,_id,textOriginal
0,665167dbc7b93965e002c573,selamat menikmati rewind indonesia di jam 19.4...
1,665167dbc7b93965e002c574,kerenn!!!\nmulai di menit ke 8 sampai akhir fi...
2,665167dbc7b93965e002c575,4.24 itu bukannya bg tanboy ya?
3,665167dbc7b93965e002c576,this is so grazy!!!:'''(
4,665167dbc7b93965e002c577,"2024 baru nonton🤚 kukira ada ""beras rose brand"""
...,...,...
34325,66516849c7b93965e0035bd1,keren banget mantap ... saking kerennya menuru...
34326,66516849c7b93965e0035bd2,kereeennn paarrraahgg
34327,66516849c7b93965e0035bd3,kok ga ada om deddy corbuzier?
34328,66516849c7b93965e0035bd5,pas nyanyi maju tak gentar merinding banget si...


#### Menghapus baris dengan nilai yang hanya berupa numeric pada kolom 'textOriginal'

In [8]:
df = df[~df['textOriginal'].str.isnumeric()]

df

,_id,textOriginal
0,665167dbc7b93965e002c573,selamat menikmati rewind indonesia di jam 19.4...
1,665167dbc7b93965e002c574,kerenn!!!\nmulai di menit ke 8 sampai akhir fi...
2,665167dbc7b93965e002c575,4.24 itu bukannya bg tanboy ya?
3,665167dbc7b93965e002c576,this is so grazy!!!:'''(
4,665167dbc7b93965e002c577,"2024 baru nonton🤚 kukira ada ""beras rose brand"""
...,...,...
34325,66516849c7b93965e0035bd1,keren banget mantap ... saking kerennya menuru...
34326,66516849c7b93965e0035bd2,kereeennn paarrraahgg
34327,66516849c7b93965e0035bd3,kok ga ada om deddy corbuzier?
34328,66516849c7b93965e0035bd5,pas nyanyi maju tak gentar merinding banget si...


#### Fungsi untuk membersihkan teks dari beberapa pola yang umum ditemukan dalam data teks

In [9]:
def clean_data_text(text):
    # Menghapus username
    text = re.sub(r'@[A-Za-z0-9_]+', '', text)
    # Menghapus hashtag
    text = re.sub(r'#\w+', '', text)
    # Menghapus angka
    text = re.sub(r'\d+', '', text)
    # Menghapus 'RT' (retweet)
    text = re.sub(r'RT[\s]+', '', text)
    # Menghapus URL
    text = re.sub(r'https?://\S+', '', text)
    # Menghapus karakter non-alfanumerik
    text = re.sub(r'[^A-Za-z0-9 ]', '', text)
    # Menghapus spasi ekstra di awal dan akhir teks, serta spasi berlebih di antara kata
    text = re.sub(r'\s+', ' ', text).strip()
    # Mencari dan menghapus karakter tunggal yang terpisah dari kata-kata
    text = re.sub(r'\b\w\b', '', text)
    # Menghapus angka tunggal
    text = re.sub(r'\b\d+\b', '', text)
    # Menghapus karakter berulang, misalnya: 'aa', 'bbb', 'ccc'
    text = re.sub(r'(.)\1+', r'\1', text)
    # Menghapus emotikon
    text = re.sub(r'(?::|;|=)(?:-)?(?:\)|\(|D|P)', '', text)
    return text

In [10]:
# Apply clean_data_text pada dataframe
# Menggunakan .loc untuk mengakses DataFrame
df.loc[:, 'textOriginal'] = df['textOriginal'].apply(clean_data_text)

df

,_id,textOriginal
0,665167dbc7b93965e002c573,selamat menikmati rewind indonesia di jam wibj...
1,665167dbc7b93965e002c574,kerenmulai di menit ke sampai akhir film asli ...
2,665167dbc7b93965e002c575,itu bukanya bg tanboy ya
3,665167dbc7b93965e002c576,this is so grazy
4,665167dbc7b93965e002c577,baru nonton kukira ada beras rose brand
...,...,...
34325,66516849c7b93965e0035bd1,keren banget mantap saking kerenya menurutku d...
34326,66516849c7b93965e0035bd2,keren parahg
34327,66516849c7b93965e0035bd3,kok ga ada om dedy corbuzier
34328,66516849c7b93965e0035bd5,pas nyanyi maju tak gentar merinding banget si...


#### Fungsi untuk membersihkan teks dari kata yang hanya 1 huruf

In [11]:
def remove_single_letter_words(text):
    return re.sub(r'\b\w\b', '', text).strip()

In [12]:
# Apply remove_single_letter_words pada dataframe
# Menggunakan .loc untuk mengakses DataFrame
df.loc[:, 'textOriginal'] = df['textOriginal'].apply(remove_single_letter_words)

df

,_id,textOriginal
0,665167dbc7b93965e002c573,selamat menikmati rewind indonesia di jam wibj...
1,665167dbc7b93965e002c574,kerenmulai di menit ke sampai akhir film asli ...
2,665167dbc7b93965e002c575,itu bukanya bg tanboy ya
3,665167dbc7b93965e002c576,this is so grazy
4,665167dbc7b93965e002c577,baru nonton kukira ada beras rose brand
...,...,...
34325,66516849c7b93965e0035bd1,keren banget mantap saking kerenya menurutku d...
34326,66516849c7b93965e0035bd2,keren parahg
34327,66516849c7b93965e0035bd3,kok ga ada om dedy corbuzier
34328,66516849c7b93965e0035bd5,pas nyanyi maju tak gentar merinding banget si...


#### Memeriksa panjang teks pada kolom 'textOriginal' dan menyimpan hanya baris dengan panjang antara 5 sampai 200 karakter

In [13]:
df = df[(df['textOriginal'].str.len() >= 5) & (df['textOriginal'].str.len() <= 200)]

df

,_id,textOriginal
1,665167dbc7b93965e002c574,kerenmulai di menit ke sampai akhir film asli ...
2,665167dbc7b93965e002c575,itu bukanya bg tanboy ya
3,665167dbc7b93965e002c576,this is so grazy
4,665167dbc7b93965e002c577,baru nonton kukira ada beras rose brand
5,665167dbc7b93965e002c578,ada faria murda
...,...,...
34325,66516849c7b93965e0035bd1,keren banget mantap saking kerenya menurutku d...
34326,66516849c7b93965e0035bd2,keren parahg
34327,66516849c7b93965e0035bd3,kok ga ada om dedy corbuzier
34328,66516849c7b93965e0035bd5,pas nyanyi maju tak gentar merinding banget si...


### Memilih hanya 10000 data sebagai sampel dari keseluruhan dataset yang telah bersih

In [14]:
# Memilih 10.000 data secara acak
df_sampled = df.sample(n=10000, random_state=42)

df_sampled

,_id,textOriginal
17254,66516815c7b93965e0030f73,kurang nih kreator gamenya ga lengkap
17282,66516815c7b93965e0030f91,wah hebat keren banget
3272,665167e7c7b93965e002d307,akting briyan furant sih
1942,665167e2c7b93965e002cd75,akhirnya bisa liat miawaug di rewind bener sih...
13535,6651680ac7b93965e002ff3a,rewind tidak berhenti di indonesia
...,...,...
19488,6651681bc7b93965e0031965,jujur ini yg paling bkin gua terharu
25577,6651682fc7b93965e003347a,mantap keren
23706,6651682ac7b93965e0032c06,keren
15436,6651680fc7b93965e0030782,semoga ada video tragedi gunung marapi sumatra...


### Normalisasi

In [15]:
norm = {
    " yg ": " yang ",
    " kren ": " keren",
    " sdh ": " sudah ",
    " dgn ": " dengan ",
    " tdk ": " tidak ",
    " dlm ": " dalam ",
    " dl ": " dulu ",
    " gw ": " saya ",
    " kok ": " kenapa ",
    " bs ": " bisa ",
    " bgt ": " sangat ",
    " njir ": " kagum ",
    " anjir ": " kagum ",
    " g ": " tidak ",
    " lg ": " lagi ",
    " dr ": " dari ",
    " tp ": " tapi ",
    " jg ": " juga ",
    " sm ": " sama ",
    " krn ": " karena ",
    " spt ": " seperti ",
    " yg ": " yang ",
    " pd ": " pada ",
    " dn ": " dan ",
    " d ": " di ",
    " kt ": " kita ",
    " km ": " kamu ",
    " ms ": " masih ",
    " tlg ": " tolong ",
    " sja ": " saja ",
    " sy ": " saya ",
    " sdikit ": " sedikit ",
    " dket ": " dekat ",
    " lbh ": " lebih ",
    " sj ": " saja ",
    " bnyk ": " banyak ",
    " dgnnya ": " dengannya ",
    " sgala ": " segala ",
    " skrg ": " sekarang ",
    " nmr ": " nomor ",
    " srg ": " sering ",
    " ttg ": " tentang ",
    " smua ": " semua ",
    " trus ": " terus ",
    " pdhl ": " padahal ",
    " kt ": " kita ",
    " dkt ": " dekat ",
    " jg ": " juga ",
    " dlm ": " dalam ",
    " bhw ": " bahwa ",
    " sja ": " saja ",
    " mrk ": " mereka ",
    " sgth ": " sangat ",
    " dn ": " dan ",
    " trlalu ": " terlalu ",
    " msing2 ": " masing-masing ",
    " brp ": " berapa ",
    " aq ": " aku ",
    " smpe ": " sampai ",
    " kalo ": " kalau ",
    " bkn ": " bukan ",
    " jd ": " jadi ",
    " dket ": " dekat ",
    " ntr ": " nanti ",
    " klo ": " kalau ",
    " sda ": " sudah ",
    " sy ": " saya ",
    " kpn ": " kapan ",
    " dmn ": " dimana ",
    " lgsng ": " langsung ",
    " tdk ": " tidak ",
    " pgi ": " pagi ",
    " ngga ": " tidak ",
    " gak ": " tidak ",
    " lbih ": " lebih ",
    " cm ": " cuman ",
    " sgt ": " sangat ",
    " aj ": " saja ",
    " apa2 ": " apa-apa ",
    " krg ": " kurang ",
    " ngerti ": " mengerti ",
    " slalu ": " selalu ",
    " bbrp ": " beberapa ",
    " smpe ": " sampai ",
    " krja ": " kerja ",
    " bbrp ": " beberapa ",
    " jln ": " jalan ",
    " hr ": " hari ",
    " kok ": " kenapa ",
    " mgkin ": " mungkin "
    # Tambahkan aturan normalisasi lainnya di sini sesuai kebutuhan
}

#### Fungsi normalisasi

In [16]:
def normalisasi(text):
    for i in norm:
        text = text.replace(i, norm[i])
    return text

In [17]:
df_sampled.loc[:, 'textOriginal'] = df_sampled['textOriginal'].apply(normalisasi)

df_sampled

,_id,textOriginal
17254,66516815c7b93965e0030f73,kurang nih kreator gamenya ga lengkap
17282,66516815c7b93965e0030f91,wah hebat keren banget
3272,665167e7c7b93965e002d307,akting briyan furant sih
1942,665167e2c7b93965e002cd75,akhirnya bisa liat miawaug di rewind bener sih...
13535,6651680ac7b93965e002ff3a,rewind tidak berhenti di indonesia
...,...,...
19488,6651681bc7b93965e0031965,jujur ini yang paling bkin gua terharu
25577,6651682fc7b93965e003347a,mantap keren
23706,6651682ac7b93965e0032c06,keren
15436,6651680fc7b93965e0030782,semoga ada video tragedi gunung marapi sumatra...


## Stopwords

In [18]:
import Sastrawi

from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary

more_stop_words = ["tidak"]

stop_words = StopWordRemoverFactory().get_stop_words()
stop_words.extend(more_stop_words)

new_array = ArrayDictionary(stop_words)
stop_words_remover_new = StopWordRemover(new_array)

def stopword(str_text):
  str_text = stop_words_remover_new.remove(str_text)
  return str_text

In [19]:
df_sampled.loc[:, 'textOriginal'] = df_sampled['textOriginal'].apply(stopword)

df_sampled

,_id,textOriginal
17254,66516815c7b93965e0030f73,kurang nih kreator gamenya ga lengkap
17282,66516815c7b93965e0030f91,wah hebat keren banget
3272,665167e7c7b93965e002d307,akting briyan furant sih
1942,665167e2c7b93965e002cd75,akhirnya liat miawaug rewind bener sih iklan h...
13535,6651680ac7b93965e002ff3a,rewind berhenti indonesia
...,...,...
19488,6651681bc7b93965e0031965,jujur yang paling bkin gua terharu
25577,6651682fc7b93965e003347a,mantap keren
23706,6651682ac7b93965e0032c06,keren
15436,6651680fc7b93965e0030782,semoga video tragedi gunung marapi sumatra bar...


## Tokenisasi
#### Memisahkan kalimat menjadi kata-kata

In [20]:
tokenized = df_sampled['textOriginal'].apply(lambda x:x.split())
tokenized

17254         [kurang, nih, kreator, gamenya, ga, lengkap]
17282                          [wah, hebat, keren, banget]
3272                         [akting, briyan, furant, sih]
1942     [akhirnya, liat, miawaug, rewind, bener, sih, ...
13535                        [rewind, berhenti, indonesia]
                               ...                        
19488            [jujur, yang, paling, bkin, gua, terharu]
25577                                      [mantap, keren]
23706                                              [keren]
15436    [semoga, video, tragedi, gunung, marapi, sumat...
16483                                      [keren, banget]
Name: textOriginal, Length: 10000, dtype: object

## Stemming
#### Mengubah kata berimbuhan menjadi kata dasar

In [21]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Inisialisasi counter
counter = 1

def stemming(text_cleaning):
    global counter
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    do = []
    for w in text_cleaning:
        dt = stemmer.stem(w)
        do.append(dt)
    d_clean = []
    d_clean = " ".join(do)
    print(f"{counter}: {d_clean}") 
    counter += 1
    return d_clean

Proses berikut agak lama karena data besar

In [22]:
tokenized = tokenized.apply(stemming)
tokenized

1: kurang nih kreator gamenya ga lengkap
2: wah hebat keren banget
3: akting briyan furant sih
4: akhir liat miawaug rewind bener sih iklan hoto ngeselin soal muncul terus siheh tomo arti baru kali rewind orang luar masuk
5: rewind henti indonesia
6: keren konsep pakai kasus david kotak pandora kasus pajak masih tega huku
7: nilai gua
8: merinding sih nontonya
9: yo merinding asa flashback ke youtube rewind dulu banyak creator lama muncul
10: pas lagu maju tak gentar nyanyi aku merinding wei
11: bu megawati nonton ya karya mileneal nih bos
12: kurang lagu somebody pleasure
13: keren moga gabung taun taun ikut
14: terima kasih banyak buat chandra liow adi victory atas karya apik sangat luar biasa
15: apasih keren sangat menit pertama aja udh bikin nang
16: keren
17: you guys doing this fxking wel
18: spechles
19: youtube rewind mah ga pernah mengecewkan
20: bagus banget
21: keren banget want crying
22: go tranding
23: best bikin merinding
24: keren banget bikin merinding nontonya
25: ke

17254                kurang nih kreator gamenya ga lengkap
17282                               wah hebat keren banget
3272                              akting briyan furant sih
1942     akhir liat miawaug rewind bener sih iklan hoto...
13535                               rewind henti indonesia
                               ...                        
19488                      jujur yang paling bkin gua haru
25577                                         mantap keren
23706                                                keren
15436    moga video tragedi gunung marapi sumatra barat...
16483                                         keren banget
Name: textOriginal, Length: 10000, dtype: object

In [23]:
tokenized

17254                kurang nih kreator gamenya ga lengkap
17282                               wah hebat keren banget
3272                              akting briyan furant sih
1942     akhir liat miawaug rewind bener sih iklan hoto...
13535                               rewind henti indonesia
                               ...                        
19488                      jujur yang paling bkin gua haru
25577                                         mantap keren
23706                                                keren
15436    moga video tragedi gunung marapi sumatra barat...
16483                                         keren banget
Name: textOriginal, Length: 10000, dtype: object

#### Mengimpan Tokenized Comments ke MongoDB Local

In [24]:
# Koneksi ke MongoDB
client = MongoClient(local_url)
db = client[database_name]
collection = db[collection_name]

# Konversi tokenized menjadi list of dictionaries
data_to_insert = [{'textOriginal': text} for text in tokenized]

# Masukkan data ke dalam koleksi MongoDB
collection.insert_many(data_to_insert)

print("Data berhasil diimpor ke MongoDB Local.")

Data berhasil diimpor ke MongoDB Local.
